### Bootstrapping data prep pipeline

This is a weakly supervised technique and should be augmented with human annotaions at a higher level. 


### Sampler function for data prep for training


- The implentation for the blocks in blue ink.
- A set of functions that takes as input :
    - CSV's at multiple levels - Image, Species count level or Image, bounding box level, etc.  
    - Sampling technique
    - number of samples required
    - this list will evolve as I write this code
    

<img src="IMG_0026.jpg" width="600" height="200">



1. Read all the shards created - ```/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/round1/bootstrap_data_<>```
2. Consolidate the list of filenames and animals
3. Sample them on the proportions that I build
4. Choose all the multi-species images

In [49]:
import pandas as pd
import csv, glob
from timeit import default_timer as timer
import numpy as np

In [53]:
start = timer()
df_base = pd.DataFrame()
for i, path in enumerate(glob.glob('/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/round1/bootstrap_data_snapshot_serengeti_s01_s06_*')):
    df = pd.read_csv(path)
    df = df[['filename', 'class']]
    df = df.drop_duplicates()
    df_base = df_base.append(df)
df_base = df_base.drop_duplicates()
end = timer()
print(i)
print(round(end - start, 1))

399
9.4


In [76]:
# get the list of multi species predictions. 
# Want to have these images in this training set. 
df_base_multi_species = df_base.groupby(by=['filename'], as_index=False)['class'].count()
df_base_multi_species.columns = ['filename', 'count_species']
df_base_multi_species = df_base_multi_species[df_base_multi_species['count_species']==2]
df_base_multi_species_lst = list(set(df_base_multi_species.filename))
df_base_multi_species.head()

,filename,count_species
434,S1/B05/B05_R3/S1_B05_R3_PICT0500,2
435,S1/B05/B05_R3/S1_B05_R3_PICT0501,2
951,S1/B07/B07_R1/S1_B07_R1_PICT0459,2
952,S1/B07/B07_R1/S1_B07_R1_PICT0460,2
1571,S1/B07/B07_R1/S1_B07_R1_PICT1219,2


In [90]:
# Sample out the image, class that are single species
# filter out the records that are not in the df_base_multi_species list
df_single_species = df_base.iloc[[val not in df_base_multi_species_lst for val in list(df_base['filename'])]]
df_single_species.head()

,filename,class
0,S5/B04/B04_R3/S5_B04_R3_IMAG0200,wildebeest
6,S5/J02/J02_R1/S5_J02_R1_IMAG0752,wildebeest
12,S5/U10/U10_R4/S5_U10_R4_IMAG3509,wildebeest
17,S3/B07/B07_R11/S3_B07_R11_IMAG0246,gazelleThomsons
18,S2/K09/K09_R1/S2_K09_R1_PICT0812,gazelleThomsons
